In [ ]:
import os
import json
import torch
from collections import defaultdict
import tensorflow as tf
import copy

In [ ]:
#합칠 json파일이 모인 위치
load_train = "./"

# 한 이미지가 가질 최대 단어수
max_output_size = 500
#iou_threshold = 0.5
iou_threshold = 0.5

#업데이트 할 annotation json 파일 이름
annotation_json = "annotation.json"
#annotation.json위치
load_path = "./"
#업데이트 한 json파일 위치와 이름
save_dir = "./ensemble.json"

In [ ]:
boxes_dict = {}
scores_dict = {}

for i in range(1):    
    with open(load_train+f"split_{i}.json", 'r', encoding='utf-8') as f:
        data = json.load(f)
        bboxes = data["images"]
        key_list_b = list(data["images"].keys())
    
    with open(load_train+f"split_{i}_s.json", 'r', encoding='utf-8') as f:
        data = json.load(f)
        scores = data["images"]
        key_list_s = list(data["images"].keys())

    key_list = list(set(key_list_b)&set(key_list_s))
    
    for key in key_list:
        if key not in boxes_dict:
            boxes_dict[key] = []

        for num in bboxes[key]["words"]:
            pnt = bboxes[key]["words"][num]["points"]
            x1= min(pnt[0][0],pnt[1][0],pnt[2][0],pnt[3][0])
            y1= min(pnt[0][1],pnt[1][1],pnt[2][1],pnt[3][1])
            x2= max(pnt[0][0],pnt[1][0],pnt[2][0],pnt[3][0])
            y2= max(pnt[0][1],pnt[1][1],pnt[2][1],pnt[3][1])
            boxes_dict[key].append([x1,y1,x2,y2])

    for key in key_list:
        if key not in scores_dict:
            scores_dict[key] = []

        for num in scores[key]["words"]:
            pnt = scores[key]["words"][num]["points"]
            score = pnt[0]
            scores_dict[key].append(score)

In [ ]:
ans_dict = {}

for key in key_list:
    if (not boxes_dict[key]) or (not scores_dict[key]):
        continue
    box = torch.Tensor(list(boxes_dict[key]))
    score = torch.Tensor(list(scores_dict[key]))
    ans =  tf.image.non_max_suppression(box, score, max_output_size, iou_threshold)
    for i in ans:
        if key not in ans_dict:
            ans_dict[key] = []
        ans_dict[key].append(boxes_dict[key][i])

nms_dict = {}
for key in ans_dict.keys():
    for x1,y1,x2,y2 in ans_dict[key]:
        point = [[x1,y1], [x2,y1], [x2,y2], [x1,y2]]
        if key not in nms_dict:
            nms_dict[key] = []
        nms_dict[key].append(point)

In [ ]:
with open(load_path + annotation_json, 'r', encoding='utf-8') as f:
    data = json.load(f)
    images = copy.deepcopy(data["images"])

    for key in images.keys():
        if key not in nms_dict:
            continue

        images[key]["words"] = {}

        for idx in range(len(ans_dict[key])):
            num = ("000" + str(idx+1))[-5:]
            x1,y1,x2,y2 = ans_dict[key][idx]

            images[key]["words"][num] = {
                "transcription" : " ",
                "point" : [[x1,y1],[x2,y1],[x2,y2],[x1,y2]],
                "orientation": "Horizontal",
                "language": None,
                "tags": [],
                "confidence": None,
                "illegibility": False }

    with open(save_dir,'w') as train_writer:
        json.dump({
            'images' : images
        }, train_writer, indent=4)

    print('\nCreating %s... Done !' %save_dir)